# Interpreting implicit feedback data for expert systems



Plan: 


0.5) Clean code of rank monitor (Work-ahead for the logging of next sprint)

1) Accurately Interpreting Clickthrough Data as Implicit Feedback (SIGIR 2016 Test Of Time Award)
	

	Trust bias   - > Bias to 'trust' engine click first reult
	Quality Bias - > If the relevance of the retrieved results decreases, 
			 users click on abstracts that are on average
		         less relevant

	Example set = S1 = {l1,. .. l7}
	
	S1 =    l1* l2 l3* l4 l5* l6 l7

	5 Strategies. 
	Click > Skip Above, 
	Last Click > Skip Above, 
	Click > Earlier Click, 
	Click > Skip Previous, 
	Click > No Click Next

2) Relative rankings to gold-set list
	2.1 Take set from SearchText that have been queried for ~200 or more times.
	2.2 Implement methodology from above paper.
	2.3 Implement graph methodology from other paper that builds on 1. 
		2.3.1 Construct graph of relevance paths. 
		2.3.2 Find largest connected graph. 

3) Delphi Questionnaires
	Does this approach match with what Delphi Questionaire would give us. 



</b>



To get a faceted search:

# Facet Search: 

http://ec2-3-120-229-133.eu-central-1.compute.amazonaws.com:8080/solr/ACC_Logging_Slave/select?facet.field=SearchText&facet.mincount=200&facet=on&indent=on&q=EventID:164%20UserID:*&rows=0&wt=jsonueyr

http://ec2-3-120-229-133.eu-central-1.compute.amazonaws.com:8080/solr/ACC_Logging_Slave/select?facet.field=SearchText&facet.mincount=200&facet=on&indent=on&q=EventID:164%20UserID:*%20AND%20ShortTimeStamp:[20181201%20TO%2020201201]&rows=0&wt=jsonueyr

De query: "unieke test query"
Aangeklikt: [3, 5, 7 17]  ->  (18-06-2019)
Aangeklikt: [1,10]        ->  (18-06-2019) +5 min later
Aangeklikt: [1,3,7]       ->  (18-06-2019) +25 min later
Aangeklikt: [2,2,7,8]     ->  (21-06-2019) ~ 15:25

De query: "Unieke test query"
TO DO Aangeklikt: [1, 10]       ->  (21-06-2019) ~ 15:33 Maar later dan



De Query: "test information retrieval"
Aangeklikt: [2, 5, 8, 26] ->  (18-06-2019)
Aangeklikt: [4, 12]       ->  (18-06-2019) + 5 min later
Aangeklikt: [1, 2, 5]     ->  (18-06-2019) +25 min later
Aangeklikt: [3, 4, 13]     ->  (21-06-2019) (15:33)

In [3]:
import numpy as np
from sklearn.metrics import f1_score, average_precision_score
import math
import requests
import os
import itertools

from urllib import *
import json

from pathlib import Path
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from collections import Counter
from tqdm import tqdm_notebook 
tqdm_notebook().pandas()
from tqdm import tqdm_notebook as tqdm

from time import strftime, gmtime
import datetime
from datetimerange import DateTimeRange
from datetime import timedelta  
from  dateutil import parser


# Greedy IDE completion 
%config IPCompleter.greedy=True

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; } </style>"))
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 30)



from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Graph-building packages
import community
import networkx as nx
import matplotlib.pyplot as plt

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

# Request dataframe from solr

In [4]:
def request_into_dataframe(rows= 100, query= '*:*', sort=''):
    """
    :param rows: amount of rows to request
    :param query: string to query
    :param sort: string to sort the request eg. sort='ShortTimeStamp desc'
    """
    
    url = 'http://ec2-3-120-229-133.eu-central-1.compute.amazonaws.com:8080/solr' \
    '/ACC_Logging_Slave/select?indent=on&q={}&rows={}&sort={}&start=0&wt=json'.format(query,rows,sort)

    print(url)
    
    response = requests.get(url)
    
    # Transform the request into a json
    response = response.json()
    response = response['response']['docs']
     
    return pd.DataFrame(response)
     
    

In [5]:
def get_27_frame(rows=100000000, from_disk=True) -> pd.DataFrame:
    """
    Get all the logging with a clicked documentPosition. 
    DocumentPosition clicked is logged since 12-2018
    :param rows: query string
    :return:
    """
    
    if from_disk:
        return pd.read_hdf('./data_rank/df27_full')
    
    else:
        df_27_bitcoin = request_into_dataframe(rows=rows, query='EventID:27 AND DocumentPosition:*')
        df_27_bitcoin.to_hdf('./data_rank/df27_full', key='test', mode='w')
        
    return df_27_bitcoin 
 

def get_16_frame(search_text = '*', rows=1000000000) -> pd.DataFrame:
    """
    Get every search log from people with a UserID:* 
    :param search_text: query string
    :param rows: number of rows to get
    :return: 
    """
    
   
    df_164_bitcoin = request_into_dataframe(query='EventID:164 AND ShortTimeStamp:[20181201 TO 20201201] AND UserID:* AND SearchText:"{}" \
                                                        '.format(search_text,search_text), rows = rows)     
    
    if search_text != search_text.lower():
        df_164_bitcoin_lowercase = request_into_dataframe(query='EventID:164 AND ShortTimeStamp:[20181201 TO 20201201] AND UserID:* AND SearchText:"{}" \
                                                            '.format(search_text.lower(),search_text.lower()), rows = rows)     
        df = pd.concat([df_164_bitcoin, df_164_bitcoin_lowercase])
    else:
        df = df_164_bitcoin
    
    
#     try:
#         # Keep only filterpath is NaN so only searches regular engine ie. no filtered searches. 
#         df = df[pd.isna(df['FilterPath'])]
#     except ValueError:
#         print('Empty dataframe')
        
    # Inster an empty column with to be filled document positions 
    df['DocumentPosition'] = df.progress_apply(lambda x: [], axis=1)
    
    df.to_hdf('./multiple_query/df164_full', key='test', mode='w')

    return df


# get_16_frame('unieke test query', rows=1000)



In [6]:
def list_currentdir():
    """
    Helper function to print contents of current directory and /data/ dir.
    """
    currentdir = Path('./data/')
    display([file for file in currentdir.glob('*')])
    display([file for file in Path('./').glob('*')])
    display([file for file in Path('./multiple_query/').glob('*')])


# list_currentdir()

In [7]:
def check_in_timerange(timestamp : datetime.datetime, query = pd.DataFrame, df_27 = pd.DataFrame) -> list:
    """
    Check if the there are clickedobjects within + and - 5 minutes from the query
    timestamp: object from the dateutil parser library
    :param timestamp:
    :param query:
    :param df_27:
    """
    
    timestamp_27 = [parser.parse(time, fuzzy_with_tokens=False) for time in df_27['TimeStamp']]
    
    doc_pos = []
    doc_ids = []
    for idx, stamp_27 in enumerate(timestamp_27):

        if timestamp in DateTimeRange(stamp_27, stamp_27 + timedelta(minutes=5)):
            
            doc_pos.append(df_27['DocumentPosition'].iloc[idx])
            doc_ids.append(df_27['DocumentID'].iloc[idx])
    
    return doc_pos, doc_ids
    

def check_and_concat(df_164: pd.DataFrame, df_27: pd.DataFrame) -> pd.DataFrame:
    """
    Check both dataframes get clicks from 27 and add these to the original query in 164
    :param df_164: dataframe containing the 164 log
    :param df_27: dataframe containing the 27 log
    """
    
    final_pos = [] 
    final_ids = []
    for idx, (df_shape, df_164_group) in enumerate(tqdm(df_164.groupby(by=['UserID','ShortTimeStamp']) , desc='Looping through queries')):
        
        # Check timestamps for dataframe 
        timestamps = [parser.parse(time, fuzzy_with_tokens=False) for time in df_164_group['TimeStamp'].tolist()]

        # Ony check searches from the specific user
        temp_27 = df_27[df_27['UserID'] == df_164_group['UserID'].iloc[0]]

        # Check every single timestamp for all the seperate events
        for stamp in timestamps:
            doc_pos, doc_ids = check_in_timerange(stamp, df_164_group, temp_27)
            final_pos.append(doc_pos)
            final_ids.append(doc_ids)

    df_164['DocumentPosition'] = final_pos
    df_164['DocumentID'] = final_ids
    
    return df_164



In [8]:
# Probabilities from Figure 1, from paper: "Generating Labels from Clicks"
probs = {1:  [1,1,0.55,0.42,0.3,0.22,0.15,0.1,0.08,0.05,0,0],
         2:  [1,1,1,0.49,0.35,0.29,0.2,0.11,0.10,0.10,0,0], 
         3:  [1,1,1,1,0.6,0.42,0.2,0.5,0.5,0.5,0,0],
         4:  [1,1,1,1,1,0.4,0.33,0.18,0.17,0.16,0,0],
         5:  [1,1,1,1,1,1,0.33,0.15,0.08,0.05,0,0],
         6:  [1,1,1,1,1,1,1,0.15,0,0,0,0],
         7:  [1,1,1,1,1,1,1,1,0.42,0.22,0,0],
         8:  [1,1,1,1,1,1,1,1,1,0.22,0,0],
         9:  [1,1,1,1,1,1,1,1,1,0.92,0,0],
         10: [1] * 12,
         11: [1] * 12,
         12: [1] * 12
         }


def get_clicked_list(df = pd.DataFrame) -> [list]:
    """
    :param df: pandas dataframe with clicked positions given query
    :return: list of clicked links 
    """

    clicks = df['DocumentPosition'].to_list()
    
    # Convert clicks to a set to leave out double clicks in a session. 
    clicks = [list(set(pos)) for pos in clicks]

    # Only keep clicks in the range 1 - 12. Ie. remove outlier clicks ie. we do not have probabilities to discount further clicks. 
    clicks = list(filter(lambda x: x != [], map(lambda x: [i for i in x if i <= 12], clicks)))
    
    return clicks
    
def make_edges_r1(df: pd.DataFrame, probs: dict) -> [tuple]:
    """
    Generate the edges to construct a graph
    It does by Rule 1: Click > skip above
    :param df:
    :param probs:
    :return: 
    """
    clicks_list = get_clicked_list(df)
    
    # From list of lists to single clicks -> treat each click as
    clicks_list = list(itertools.chain.from_iterable(clicks_list))
    edges = []
    for click in clicks_list:
        dummy_probabilities = []
        for i in range(1,12):
            if i != click and probs[click][i] > 0:
                dummy_probabilities.append((i,click, probs[click][i]))
        edges.extend(dummy_probabilities)
    display(edges)
        
    return edges

def make_edges_r2(df: pd.DataFrame, probs: dict) -> [tuple]:
    """
    Generate the edges to construct a graph
    It does by Rule 2: Last click > skip above
    :param df:
    :param probs:
    :return: 
    """
    clicks_list = get_clicked_list(df)
    
    # From list of lists to single clicks -> treat each click as
    edges = []
    for clicks in clicks_list:
        # Takes last click in clicks list
        click = clicks[-1]
        dummy_probabilities = []

        for i in range(1, 12):
            if i not in clicks and i != click and probs[click][i] > 0:
                  dummy_probabilities.append((i,click, probs[click][i]))
        edges.extend(dummy_probabilities)
    
    return edges

def make_edges_r3(df: pd.DataFrame, probs: dict) -> [tuple]:
    """
    Generate the edges to construct a graph
    NOT PROBABILISITC
    It does by Rule 3: Cick > Earlier Click
    :param df:
    :param probs:
    :return: 
    """
    # From list of lists to single clicks -> treat each click as
    edges = []
    for clicks in clicks_list:

        dummy_probabilities = []

        # Takes clicks from position 2 forward
        for idx, click in enumerate(clicks):
            if idx > 0:
                ranges = np.arange(0,idx)
                for i in ranges:
                    dummy_probabilities.append((clicks[i], clicks[idx], 1))

        edges.extend(dummy_probabilities)

    return edges


def make_edges_r4(df: pd.DataFrame, probs: dict) -> [tuple]:
    """
    Generate the edges to construct a graph
    NOT PROBABILISITC
    It does by Rule 3: Cick > Earlier Click
    :param df:
    :param probs:
    :return: 
    """
    # From list of lists to single clicks -> treat each click as
    edges = []
    for clicks in clicks_list:

        dummy_probabilities = []

        # Takes clicks from position 2 forward
        for idx, click in enumerate(clicks):
            if idx > 0:
                ranges = np.arange(0,idx)
                for i in ranges:
                    dummy_probabilities.append((clicks[i], clicks[idx], 1))

        edges.extend(dummy_probabilities)

    return edges


In [9]:
def create_relevance_graph(edges: [tuple]):
    """
    Helper function to create a relevance graph between relevant documents. 
    :param edges: lis of edges between two documents and their signal strength
    """
    DG = nx.DiGraph()
    DG.add_weighted_edges_from(edges)

    # Forcing a layout 
    pos = nx.spring_layout(DG)

    # Make a label dictionary: {} for labeling of graph 
    new_labels = dict(map(lambda x: ((x[0],x[1]), x[2]['weight']), DG.edges(data=True)))
    
    nx.draw_networkx(DG, pos=pos,alpha=0.5)
    nx.draw_networkx_edge_labels(DG, pos=pos, edge_labels=new_labels)
    nx.draw_networkx_edge_labels(DG, pos=pos)
    plt.title('Pair-wise network')
    plt.savefig('Large-graph')
    plt.show()

    dicto = nx.pagerank(DG)

    print("Before sorting: \n", dicto) 
    dicto = sorted(dicto.items(), key=lambda x: x[1], reverse=True)
    print("After sorting: \n", dicto)

        


In [11]:
def start_search(query_str='AVG') -> pd.DataFrame:
    """
    :param: query_str query string to request clicks for 
    :return: DataFrame to 
    """
    
    print("Get dataframes: ")
    df_27 = get_27_frame(rows=10000000,from_disk=True)
    print("Got dataframe 27")
    df_164 = get_16_frame(search_text= query_str, rows=10000)
    print("Loaded both frames")
    
    display(df_27)
    display(df_164)

    df = check_and_concat(df_164=df_164, df_27=df_27)
    
    # Remove events that did not have a single click
    df = df[df['DocumentPosition'].str.len() != 0]
    df = df[pd.isna(df['FilterPath'])]

    
    print("shape dataframe", df.shape)
    
    # Make edges & construct relevance graph 
    edges = make_edges_r1(df, probs)
    
    # Display some edges
    edges[0][:10]
    create_relevance_graph(edges=edges)

    return df


df = start_search()
df

# df['DocumentPosition'].tolist()

# number = df.progress_apply(lambda x: [{click : [docid]} for click, docid in zip(df['DocumentPosition'].tolist(), df['DocumentID'].tolist())] , axis=1)

# df.to_hdf('./multiple_query/df27_{}'.format('Full concat'), key='test', mode='w')


Get dataframes: 
Got dataframe 27
http://ec2-3-120-229-133.eu-central-1.compute.amazonaws.com:8080/solr/ACC_Logging_Slave/select?indent=on&q=EventID:164 AND ShortTimeStamp:[20181201 TO 20201201] AND UserID:* AND SearchText:"AVG"                                                         &rows=10000&sort=&start=0&wt=json


ConnectionError: HTTPConnectionPool(host='ec2-3-120-229-133.eu-central-1.compute.amazonaws.com', port=8080): Max retries exceeded with url: /solr/ACC_Logging_Slave/select?indent=on&q=EventID:164%20AND%20ShortTimeStamp:[20181201%20TO%2020201201]%20AND%20UserID:*%20AND%20SearchText:%22AVG%22%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20&rows=10000&sort=&start=0&wt=json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fe8bd7cad68>: Failed to establish a new connection: [Errno 110] Connection timed out',))

In [ ]:
df_27 = get_27_frame(rows=10000000,from_disk=True)
df_27 = df_27[(df_27['SearchText'] == 'AVG' ) | (df_27['SearchText'] == 'avg' )]

   
new_cols = ['UserID', 'DocumentID', 'DocumentPosition', 'DocumentURL', 'EventID', 'ID',
                   'SearchTypeID', 'ShortTimeStamp', 'TimeStamp', 'SearchText']

df_27 = df_27[new_cols]

df_27.sort_values(by=['DocumentPosition'])
df_27

In [ ]:
for i in tqdm(range(1,1000)):
    pass